# Human Preference Prediction Baseline

This notebook implements a simple baseline model for predicting human preference of LLM responses using lexical and length-based features with Logistic Regression.

## 1. Load Data

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import re
import lightgbm

path = "/kaggle/input/llm-classification-finetuning/"           #for kaggle submission
#path = "dataset/"                                              #for local test
train_df = pd.read_csv('dataset/train.csv')
test_df = pd.read_csv('dataset/test.csv')
submission_df = pd.read_csv('dataset/sample_submission.csv')

train_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'dataset/train.csv'

## 2. Feature Engineering

In [2]:
def get_features(df):
    # Length features
    df['len_a'] = df['response_a'].str.len()
    df['len_b'] = df['response_b'].str.len()
    df['word_count_a'] = df['response_a'].apply(lambda x: len(str(x).split()))
    df['word_count_b'] = df['response_b'].apply(lambda x: len(str(x).split()))
    
    # Sentence count features
    df['sent_count_a'] = df['response_a'].apply(lambda x: len(re.findall(r'.', str(x))))
    df['sent_count_b'] = df['response_b'].apply(lambda x: len(re.findall(r'.', str(x))))
    
    # Punctuation count features
    df['punc_count_a'] = df['response_a'].apply(lambda x: len(re.findall(r'[!?,;:]', str(x))))
    df['punc_count_b'] = df['response_b'].apply(lambda x: len(re.findall(r'[!?,;:]', str(x))))
    
    # Feature differences
    df['len_diff'] = df['len_a'] - df['len_b']
    df['word_count_diff'] = df['word_count_a'] - df['word_count_b']
    df['sent_count_diff'] = df['sent_count_a'] - df['sent_count_b']
    df['punc_count_diff'] = df['punc_count_a'] - df['punc_count_b']
    return df

train_df = get_features(train_df)
test_df = get_features(test_df)

## 3. Model Training

In [7]:
# Create target variable
conditions = [
    train_df['winner_model_a'] == 1,
    train_df['winner_model_b'] == 1,
    train_df['winner_tie'] == 1
]
choices = [0, 1, 2] # 0: model_a, 1: model_b, 2: tie
train_df['winner'] = np.select(conditions, choices, default=-1)

features = [
    'len_a', 'len_b', 'word_count_a', 'word_count_b', 'sent_count_a', 'sent_count_b',
    'punc_count_a', 'punc_count_b', 'len_diff', 'word_count_diff', 'sent_count_diff', 'punc_count_diff'
]

X_train = train_df[features]
y_train = train_df['winner']
X_test = test_df[features]

model = LogisticRegression(max_iter=10000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=10000)

## 4. Prediction

In [8]:
predictions = model.predict_proba(X_test)

## 5. Submission

In [9]:
submission_df['winner_model_a'] = predictions[:, 0]
submission_df['winner_model_b'] = predictions[:, 1]
submission_df['winner_tie'] = predictions[:, 2]

submission_df.to_csv('submission.csv', index=False)

submission_df.head()

,id,winner_model_a,winner_model_b,winner_tie
0,136060,0.327406,0.337742,0.334852
1,211333,0.429564,0.253238,0.317198
2,1233961,0.432636,0.286542,0.280822
